In [1]:
import pandas as pd
import numpy as np
import gc

In [2]:
def one_hot_encoding(df, nan_as_category=True):
    original_columns = list(df.columns)
    categorical_columns = [
        col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns=categorical_columns,
                        dummy_na=nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns

def credit_card_balance(num_rows=None, nan_as_category=True):
    cc = pd.read_csv('../input/credit_card_balance.csv', nrows=num_rows)
    # number of loans
    cc_agg = cc.groupby('SK_ID_CURR')['SK_ID_PREV'].\
                nunique().\
                reset_index().\
                rename(index = str, columns = {'SK_ID_PREV': 'CC_USR_LOAN_CNT'})
    cc_agg.set_index('SK_ID_CURR', inplace=True)
    # number of credit balance records
    cc_agg['CC_BLANCE_REC_CNT'] = cc.groupby('SK_ID_CURR').size()
    cc_agg['CC_PAYBACK_TIMES_MAX'] = cc.groupby('SK_ID_CURR')['CNT_INSTALMENT_MATURE_CUM'].max()
    
    # handle on payback times
    temp = cc.groupby(['SK_ID_PREV', 'SK_ID_CURR'])['CNT_INSTALMENT_MATURE_CUM'].\
                max().\
                reset_index().\
                rename(index=str, columns={'CNT_INSTALMENT_MATURE_CUM': 'INSTALLMENT_TIMES_PER_LOAN'})
    cc_agg['CC_PAYBACK_TIMES_TOTAL'] = temp.groupby('SK_ID_CURR')['INSTALLMENT_TIMES_PER_LOAN'].sum()
    cc_agg['CC_AVG_PAYBACK_TIMES'] = cc_agg['CC_PAYBACK_TIMES_TOTAL'] / cc_agg['CC_USR_LOAN_CNT']
    
    # handle on DPD
    cc_agg['CC_DPD_MAX'] = cc.groupby('SK_ID_CURR')['SK_DPD'].max()
    cc['SK_DPD_GT_ZERO'] = cc['SK_DPD'] > 0
    temp = cc.groupby(['SK_ID_PREV', 'SK_ID_CURR'])['SK_DPD_GT_ZERO'].\
                sum().\
                reset_index().\
                rename(index=str, columns={'SK_DPD_GT_ZERO': 'CNT_DPD_PER_LOAN'})
    cc_agg['CC_USR_AVG_DPD_CNT'] = temp.groupby('SK_ID_CURR')['CNT_DPD_PER_LOAN'].mean()
    cc_agg['CC_USR_AVG_DPD'] = cc.groupby('SK_ID_CURR')['SK_DPD'].mean()    

    # handle on DPD DEF
    cc_agg['CC_DPD_DEF_MAX'] = cc.groupby('SK_ID_CURR')['SK_DPD_DEF'].max()
    cc['SK_DPD_DEF_GT_ZERO'] = cc['SK_DPD_DEF'] > 0
    temp = cc.groupby(['SK_ID_PREV', 'SK_ID_CURR'])['SK_DPD_DEF_GT_ZERO'].\
                sum().\
                reset_index().\
                rename(index=str, columns={'SK_DPD_DEF_GT_ZERO': 'CNT_DPD_DEF_PER_LOAN'})
    cc_agg['CC_USR_AVG_DPD_DEF_CNT'] = temp.groupby('SK_ID_CURR')['CNT_DPD_DEF_PER_LOAN'].mean()
    cc_agg['CC_USR_AVG_DPD_DEF'] = cc.groupby('SK_ID_CURR')['SK_DPD_DEF'].mean()
    
    # the ratio of minimum installment missed
#     cc_agg['CC_USR_MINIMUM_PAYMENT_MISS_RATIO'] = \
#         cc.groupby('SK_ID_CURR')[['AMT_PAYMENT_CURRENT', 'AMT_INST_MIN_REGULARITY']].\
#         apply(lambda x: (cc['AMT_PAYMENT_CURRENT'] < cc['AMT_INST_MIN_REGULARITY']).sum() / len(cc))
    
    cc['RECIVABLE_TO_PRINCIPAL_RATIO'] = cc['AMT_RECIVABLE'] / cc['AMT_RECEIVABLE_PRINCIPAL']
    cc['RECIVABLE_TO_TOTAL_RATIO'] = cc['AMT_RECIVABLE'] / cc['AMT_TOTAL_RECEIVABLE']
    cc['BALANCE_TO_CREDIT_RATIO'] = cc['AMT_BALANCE'] / cc['AMT_CREDIT_LIMIT_ACTUAL']
    cc['PAYBACK_LT_INST_MIN'] = cc['AMT_PAYMENT_CURRENT'] < cc['AMT_INST_MIN_REGULARITY']
    cc['PAYBACK_TO_INST_MIN_RATIO'] = cc['AMT_PAYMENT_CURRENT'] / cc['AMT_INST_MIN_REGULARITY']
    
    num_aggregations = {
        # AMT_DRAWINGS
        'AMT_DRAWINGS_ATM_CURRENT': ['max', 'mean', 'sum'],
        'AMT_DRAWINGS_CURRENT': ['max', 'mean', 'sum'],
        'AMT_DRAWINGS_OTHER_CURRENT': ['max', 'mean', 'sum'],
        'AMT_DRAWINGS_POS_CURRENT': ['max', 'mean', 'sum'],

        # CNT_DRAWINGS
        'CNT_DRAWINGS_ATM_CURRENT': ['max', 'mean', 'sum'],
        'CNT_DRAWINGS_CURRENT': ['max', 'mean', 'sum'],
        'CNT_DRAWINGS_OTHER_CURRENT': ['max', 'mean', 'sum'],
        'CNT_DRAWINGS_POS_CURRENT': ['max', 'mean', 'sum'],
        
        # OTHER AMT & RATIO
        'AMT_BALANCE': ['max', 'mean', 'std'],
        'AMT_CREDIT_LIMIT_ACTUAL': ['max', 'mean', 'std'],
        'BALANCE_TO_CREDIT_RATIO': ['max', 'mean', 'std'],
        'RECIVABLE_TO_PRINCIPAL_RATIO': ['max', 'mean', 'std'],
        'RECIVABLE_TO_TOTAL_RATIO': ['max', 'mean', 'std'],
        'PAYBACK_LT_INST_MIN': ['mean', 'sum'],
        'PAYBACK_TO_INST_MIN_RATIO': ['max', 'mean', 'std'],
    }
    
    cc, cat_cols = one_hot_encoding(cc, nan_as_category=True)
    
    cate_aggregations = {}
    for cat in cat_cols:
        cate_aggregations[cat] = ['mean']
    
    cc_agg_auto = cc.groupby('SK_ID_CURR').agg({**num_aggregations, **cate_aggregations})
    cc_agg_auto.columns = pd.Index(['CC_' + e[0] + "_" + e[1].upper() for e in cc_agg_auto.columns.tolist()])
    
    # SOME ADDITIONAL OPERATION
    cc_agg_auto['CC_DRAWINGS_AMT_ATM_RATIO'] = cc_agg_auto['CC_AMT_DRAWINGS_ATM_CURRENT_SUM'] \
                                        / cc_agg_auto['CC_AMT_DRAWINGS_CURRENT_SUM']
    cc_agg_auto['CC_DRAWINGS_AMT_OTHER_RATIO'] = cc_agg_auto['CC_AMT_DRAWINGS_OTHER_CURRENT_SUM'] \
                                        / cc_agg_auto['CC_AMT_DRAWINGS_CURRENT_SUM']
    cc_agg_auto['CC_DRAWINGS_AMT_POS_RATIO'] = cc_agg_auto['CC_AMT_DRAWINGS_POS_CURRENT_SUM'] \
                                        / cc_agg_auto['CC_AMT_DRAWINGS_CURRENT_SUM']
    
    cc_agg_auto['CC_DRAWINGS_CNT_ATM_RATIO'] = cc_agg_auto['CC_CNT_DRAWINGS_ATM_CURRENT_SUM'] \
                                        / cc_agg_auto['CC_CNT_DRAWINGS_CURRENT_SUM']
    cc_agg_auto['CC_DRAWINGS_CNT_OTHER_RATIO'] = cc_agg_auto['CC_CNT_DRAWINGS_OTHER_CURRENT_SUM'] \
                                        / cc_agg_auto['CC_CNT_DRAWINGS_CURRENT_SUM']
    cc_agg_auto['CC_DRAWINGS_CNT_POS_RATIO'] = cc_agg_auto['CC_CNT_DRAWINGS_POS_CURRENT_SUM'] \
                                        / cc_agg_auto['CC_CNT_DRAWINGS_CURRENT_SUM']

    cc_agg = cc_agg.join(cc_agg_auto)
    del cc, temp, cc_agg_auto
    gc.collect()
    return cc_agg
cc_agg = credit_card_balance()

In [3]:
cc_agg

,CC_USR_LOAN_CNT,CC_BLANCE_REC_CNT,CC_PAYBACK_TIMES_MAX,CC_PAYBACK_TIMES_TOTAL,CC_AVG_PAYBACK_TIMES,CC_DPD_MAX,CC_USR_AVG_DPD_CNT,CC_USR_AVG_DPD,CC_DPD_DEF_MAX,CC_USR_AVG_DPD_DEF_CNT,...,CC_NAME_CONTRACT_STATUS_Refused_MEAN,CC_NAME_CONTRACT_STATUS_Sent proposal_MEAN,CC_NAME_CONTRACT_STATUS_Signed_MEAN,CC_NAME_CONTRACT_STATUS_nan_MEAN,CC_DRAWINGS_AMT_ATM_RATIO,CC_DRAWINGS_AMT_OTHER_RATIO,CC_DRAWINGS_AMT_POS_RATIO,CC_DRAWINGS_CNT_ATM_RATIO,CC_DRAWINGS_CNT_OTHER_RATIO,CC_DRAWINGS_CNT_POS_RATIO
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100006,1,6,0.0,0.0,0.0,0,0.0,0.000000,0,0.0,...,0.0,0.0,0.000000,0,NaN,NaN,NaN,NaN,NaN,NaN
100011,1,74,33.0,33.0,33.0,0,0.0,0.000000,0,0.0,...,0.0,0.0,0.000000,0,1.000000,0.000000,0.000000,1.000000,0.00000,0.000000
100013,1,96,22.0,22.0,22.0,1,1.0,0.010417,1,1.0,...,0.0,0.0,0.000000,0,1.000000,0.000000,0.000000,1.000000,0.00000,0.000000
100021,1,17,0.0,0.0,0.0,0,0.0,0.000000,0,0.0,...,0.0,0.0,0.000000,0,NaN,NaN,NaN,NaN,NaN,NaN
100023,1,8,0.0,0.0,0.0,0,0.0,0.000000,0,0.0,...,0.0,0.0,0.000000,0,NaN,NaN,NaN,NaN,NaN,NaN
100028,1,49,35.0,35.0,35.0,0,0.0,0.000000,0,0.0,...,0.0,0.0,0.000000,0,0.089504,0.000000,0.910496,0.017094,0.00000,0.982906
100036,1,12,0.0,0.0,0.0,0,0.0,0.000000,0,0.0,...,0.0,0.0,0.000000,0,NaN,NaN,NaN,NaN,NaN,NaN
100042,1,84,39.0,39.0,39.0,1,2.0,0.023810,0,0.0,...,0.0,0.0,0.000000,0,0.533554,0.417863,0.048583,0.593750,0.15625,0.250000
100043,1,33,20.0,20.0,20.0,0,0.0,0.000000,0,0.0,...,0.0,0.0,0.000000,0,0.137233,0.000000,0.862767,0.166667,0.00000,0.833333


In [6]:
cc_agg.CC_PAYBACK_LT_INST_MIN_MEAN

SK_ID_CURR
100006    0.000000
100011    0.000000
100013    0.010417
100021    0.000000
100023    0.000000
100028    0.020408
100036    0.000000
100042    0.023810
100043    0.212121
100047    0.000000
100048    0.000000
100049    0.000000
100050    0.000000
100059    0.000000
100066    0.000000
100067    0.034483
100075    0.000000
100077    0.000000
100082    0.068182
100083    0.000000
100086    0.047619
100090    0.000000
100100    0.117021
100101    0.071429
100107    0.181818
100109    0.037975
100110    0.000000
100112    0.000000
100114    0.000000
100115    0.000000
            ...   
456173    0.062500
456178    0.000000
456186    0.076923
456188    0.000000
456189    0.217391
456195    0.000000
456198    0.083333
456199    0.000000
456201    0.000000
456206    0.000000
456210    0.000000
456211    0.000000
456213    0.029412
456214    0.000000
456215    0.000000
456217    0.021053
456219    0.000000
456225    0.125000
456233    0.000000
456235    0.000000
456236    0.010638
4